In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import hsv_to_rgb

# Import Data
df_apt = pd.read_csv('4_Map_Data.csv')
df_geo = pd.read_csv('2_Final_Geospatial_data.csv')



ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

### Map of geospacial data points:

In [ ]:
# Define a color mapping based on the Type
type_color_mapping = {
    "Malls": "blue",
    "Hospitals": "green",
    "Restaurants": "purple",
    "Schools": "red",
    "Beaches": "yellow",
    "Public Transport Stations": "orange"
}

# Create a map focused on Denmark
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='merc', llcrnrlat=54, urcrnrlat=58, llcrnrlon=8, urcrnrlon=15, resolution='f')
m.drawcountries()
m.drawmapboundary(fill_color='#A9A9A9')  # Darker grey for water
m.fillcontinents(color='#D3D3D3',lake_color='#A9A9A9')  # Light grey for land

# Plot the data points on the map
for idx, row in df_geo.iterrows():
    color = type_color_mapping.get(row["Type"], "black")
    x, y = m(row["Longitude"], row["Latitude"])
    m.plot(x, y, 'o', color=color, markersize=2)

# Create a custom legend
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=type_) for type_, color in type_color_mapping.items()]
plt.legend(handles=legend_elements, loc='lower right')

# Save as png
plt.savefig("Map of geospacial data points.png", dpi=300)

plt.show()


### Map of apartments data points:

In [ ]:
# Define a function to map colors based on postal code
def get_color(postal_code):
    if 1000 <= postal_code < 2000:
        hue = 0.1111*0.1
    elif 2000 <= postal_code < 3000:
        hue = 0.2222*0.2
    elif 3000 <= postal_code < 4000:
        hue = 0.3333*0.2
    elif 4000 <= postal_code < 5000:
        hue = 0.4444*0.25
    elif 5000 <= postal_code < 6000:
        hue = 0.5555*0.3
    elif 6000 <= postal_code < 7000:
        hue = 0.6666*0.4
    elif 7000 <= postal_code < 8000:
        hue = 0.7777*0.45
    elif 8000 <= postal_code < 9000:
        hue = 0.8888*0.5
    else:
        hue = 0.9999*0.55
    
    saturation = 1.0
    brightness = 0.9
    
    hsv_color = (hue, saturation, brightness)
    rgb_color = hsv_to_rgb(hsv_color)
    
    return tuple(rgb_color)


# Create a map focused on Denmark
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='merc', llcrnrlat=54, urcrnrlat=58, llcrnrlon=8, urcrnrlon=15, resolution='f')
m.drawcountries()
m.drawmapboundary(fill_color='#A9A9A9')  # Darker grey for water
m.fillcontinents(color='#D3D3D3', lake_color='#A9A9A9')  # Light grey for land

# Plot the data points on the map
for idx, row in df_apt.iterrows():
    color = get_color(row["Postal code"])  # Use the correct column name
    x, y = m(row["Longitude"], row["Latitude"])
    m.plot(x, y, 'o', color=color, markersize=2)

# Add a legend for the postal code ranges
labels = ["1000-1999", "2000-2999", "3000-3999", "4000-4999", "5000-5999", "6000-6999", "7000-7999", "8000-8999", "9000+"]
colors = [get_color(code * 1000 + 1000) for code in range(1, 10)]
patches = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) for label, color in zip(labels, colors)]

# Adjust legend placement
plt.legend(handles=patches, title="Postal Code Ranges", loc='lower right', bbox_to_anchor=(0.99, 0.01), borderaxespad=0.)

# Save as png
plt.savefig("Map of apartments data points.png", dpi=300)

plt.show()


### Map of apartment and distance from geospacial data:

In [ ]:
# Define a color mapping based on the Type
type_color_mapping = {
    "Malls": "blue",
    "Hospitals": "green",
    "Restaurants": "purple",
    "Schools": "red",
    "Beaches": "yellow",
    "Public Transport Stations": "orange"
}

# Define a color mapping for closest points
closest_point_color_mapping = {
    'Closest Transport Station': "orange",
    'Closest Beach': "yellow",
    'Closest School': "red",
    'Closest Restaurant': "purple",
    'Closest Hospital': "green",
    'Closest Mall': "blue"
}

# Create a map focused on Denmark
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='merc', llcrnrlat=56.0, urcrnrlat=56.4, llcrnrlon=10.1, urcrnrlon=10.8, resolution='f')
m.drawcountries()
m.drawmapboundary(fill_color='#A9A9A9')  # Darker grey for water
m.fillcontinents(color='#D3D3D3', lake_color='#A9A9A9')  # Light grey for land

# Plot the data points on the map
for idx, row in df_geo.iterrows():
    color = type_color_mapping.get(row["Type"])
    x, y = m(row["Longitude"], row["Latitude"])
    m.plot(x, y, 'o', color=color, markersize=2)

# Retrieve and plot the related points
related_points = [
    ('Closest Transport Station', 'closest_Public Transport Stations_longitude', 'closest_Public Transport Stations_latitude'),
    ('Closest Beach', 'closest_Beaches_longitude', 'closest_Beaches_latitude'),
    ('Closest School', 'closest_Schools_longitude', 'closest_Schools_latitude'),
    ('Closest Restaurant', 'closest_Restaurants_longitude', 'closest_Restaurants_latitude'),
    ('Closest Hospital', 'closest_Hospitals_longitude', 'closest_Hospitals_latitude'),
    ('Closest Mall', 'closest_Malls_longitude', 'closest_Malls_latitude')
]

for point_type, lon_col, lat_col in related_points:
    lon = df_apt.loc[5541, lon_col]
    lat = df_apt.loc[5541, lat_col]
    if point_type in type_color_mapping:
        color = type_color_mapping[point_type]
    elif point_type in closest_point_color_mapping:
        color = closest_point_color_mapping[point_type]
    else:
        color = "black"  # Default color if not found
    x, y = m(lon, lat)
    m.plot(x, y, 'o', color=color, markersize=6, markeredgecolor='black', markeredgewidth=1.5)

    # Retrieve coordinates of the new apartment and related points (index 5541)
    apt_coords = df_apt.loc[5541, ["Longitude", "Latitude"]]
    apt_color = type_color_mapping.get("Apartments", "cyan")

    # Plot the new apartment on the map with a larger size and black outline
    apt_x, apt_y = m(apt_coords["Longitude"], apt_coords["Latitude"])
    m.plot(apt_x, apt_y, 'o', color=apt_color, markersize=6, markeredgecolor='black', markeredgewidth=1.5)

    # Plot grey dotted lines from apartment to closest points
    m.plot([apt_x, x], [apt_y, y], color='grey', linestyle='dotted')

# Create a custom legend
legend_elements = []

# Add legend element for the new apartment
legend_elements.append(Line2D([0], [0], marker='o', color='none', markerfacecolor=apt_color, markeredgecolor='black', markersize=10, markeredgewidth=1.5, label="Apartment"))

# Add legend elements for type_color_mapping
for type_, color in type_color_mapping.items():
    legend_elements.append(Line2D([0], [0], marker='o', color='none', markerfacecolor=color, markersize=10, label=type_))

# Add legend elements for related points
for point_type, _, _ in related_points:
    if point_type in closest_point_color_mapping:
        color = closest_point_color_mapping[point_type]
        legend_elements.append(Line2D([0], [0], marker='o', color='none', markerfacecolor=color, markeredgecolor='black', markersize=10, markeredgewidth=1.5, label=point_type))

plt.legend(handles=legend_elements, loc='lower right')

# Save as png
plt.savefig("Map of apartment and distance from geospacial data.png", dpi=300)

plt.show()
